In [6]:
from os import listdir
import pandas
import re

from pg import DB
db = DB()

In [3]:
sizes = []
for sample_file_name in sample_file_names:
    size = int(re.search(r'\d+', sample_file_name).group())
    sizes.append(size)



### Creating database table to store sample scores

In [4]:
db.query("""create table samples_scores(
         size integer,
         average_score float,
         tt_average_score float
        )""")

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 9))



ProgrammingError: ERROR:  relation "samples_scores" already exists


### Inserting samples scores into database

In [ ]:
%%time
sample_file_names = listdir("files/intermediates/samples/")
for sample_file_name in sample_file_names:
    print(sample_file_name)
    size = int(re.search(r'\d+', sample_file_name).group())
    sample = pandas.read_pickle(open( "files/intermediates/samples/"+sample_file_name, "rb" ))
    sample_df =pandas.DataFrame(sample)
    pandas.DataFrame(sample)
    for i, row in enumerate(sample_df["average_scores"]):
        do_not_print = db.insert('samples_scores',
                             size = size,
                             average_score = sample_df["average_scores"][i],
                             tt_average_score = sample_df["tt_average_scores"][i]
                            )

### Creating database table to store motif scores

In [ ]:
db.query("""create table motif_average_scores(
         motif_string varchar primary key,
         proteins_length integer,
         average_score float,
         tt_average_score float,
         shuffled_proteins_length integer,
         shuffled_average_score float,
         tt_shuffled_average_score float
        )""")

### Inserting motif scores into database

In [ ]:
%%time
for motif_scores_file_name in motif_scores_file_names:
    print(motif_scores_file_name)
    motif_scores = pandas.read_pickle(open( "files/motif_scores/"+motif_scores_file_name, "rb" ))
    motif_scores_df =pandas.DataFrame(motif_scores)
#     pandas.DataFrame(sample)
    for i, row in enumerate(motif_scores_df["average_scores"]):
#         print(motif_scores_df.index[i], motif_scores_df["motif_subgraph_size"][i], motif_scores_df["average_scores"][i])
        do_not_print = db.insert('motif_average_scores',
                             motif_string = motif_scores_df.index[i],
                             proteins_length = motif_scores_df["motif_subgraph_size"][i],
                             average_score = motif_scores_df["average_scores"][i],
                             tt_average_score = motif_scores_df["average_tt_scores"][i],
                             shuffled_proteins_length = motif_scores_df["motif_shuffled_subgraph_size"][i],
                             shuffled_average_score = motif_scores_df["shuffled_average_scores"][i],
                             tt_shuffled_average_score = motif_scores_df["shuffled_average_tt_scores"][i]
                            )

### Adding collumns for p_values into the motifs scores table

In [ ]:
db.query("ALTER TABLE motif_average_scores ADD COLUMN p_value float;")
db.query("ALTER TABLE motif_average_scores ADD COLUMN shuffled_p_value float;")
db.query("ALTER TABLE motif_average_scores ADD COLUMN tt_p_value float;")
db.query("ALTER TABLE motif_average_scores ADD COLUMN tt_shuffled_p_value float;")

### Updating p-values for motifs

In [ ]:
%%time
p_values_file_names = listdir("files/intermediates/p_values/")
for p_values_file in p_values_file_names:
    p_values = pandas.read_pickle(open( "files/intermediates/p_values/"+p_values_file, "rb" ))
    p_values_df =pandas.DataFrame(p_values)
    for i, row in enumerate(p_values_df.index):
        do_not_print = db.query("UPDATE motif_average_scores SET p_value = '" + str(p_values_df['motif_p_value'][row]) + "', shuffled_p_value = '" + str(p_values_df['motif_shuffled_p_value'][row]) + "', tt_p_value = '" + str(p_values_df['motif_tt_p_value'][row]) + "', tt_shuffled_p_value = '" +str(p_values_df['motif_shuffled_tt_p_value'][row]) +  "' WHERE motif_string  = '" + str(row) + "'")



Peeking into p_values. 

NaN values represent entries where the p-value was not calculated because the score was not calculated since the subgraph was bigger than our cut off size. If both the subgraph and subgraph of the shuffled results are bigger than cutoff value the motif is not even in the table below.

In [11]:
p_values_df

,motif_p_value,motif_shuffled_p_value,motif_shuffled_tt_p_value,motif_tt_p_value
..[ag]tcgcg,0.0140,NaN,NaN,1.0
..cgctac,0.1671,NaN,NaN,1.0
..cggtac,0.0088,NaN,NaN,1.0
..ctagcg,0.0251,NaN,NaN,1.0
..gcgcgc,0.3642,NaN,NaN,1.0
.[ag][ct]cggta,0.0163,NaN,NaN,1.0
.[ag]acatcg,0.0666,0.2174,1.0,1.0
.[ag]accgca,0.1563,0.1807,1.0,1.0
.[ag]acggac,0.0012,0.2563,1.0,1.0
.[ag]atgtcg,0.0255,0.2739,1.0,1.0
